In [1]:
import requests
import pandas as pd

### DATA SOURCE: 

* Railway Group Data:
[IBB](https://data.ibb.gov.tr/en/dataset/metro-istanbul-rayli-sistem-grup-listesi) --> [Metro Istanbul Rail System Group List](https://data.ibb.gov.tr/en/dataset/metro-istanbul-rayli-sistem-grup-listesi/resource/556a2d81-d1aa-4bf7-9174-8be310116d4d) --> [API data](https://api.ibb.gov.tr/MetroIstanbul/api/MetroMobile/V2/GetRailwayGroups)

* Metro Station Data:
[IBB](https://data.ibb.gov.tr/en/dataset/metro-istanbul-hat-bilgisi-listesi) --> [Metro İstanbul Web Services Help Page](https://data.ibb.gov.tr/en/dataset/metro-istanbul-hat-bilgisi-listesi/resource/2755291e-a7ba-4be9-a00d-1cbaf6b303a1) --> [Metro Istanbul API's list](https://api.ibb.gov.tr/MetroIstanbul/Help) --> [GET api/MetroMobile/GetStations](https://api.ibb.gov.tr/MetroIstanbul/Help/Api/GET-api-MetroMobile-GetStations)


## Railway Group List

In [2]:
railway_groups_api = "https://api.ibb.gov.tr/MetroIstanbul/api/MetroMobile/V2/GetRailwayGroups"
r = requests.get(railway_groups_api)
print(r)
railway_groups_json_obj = r.json()

for row in railway_groups_json_obj['Data']:
    print(row)
    print()

<Response [200]>
{'Group': 'M', 'Name': 'Metro Hatları', 'Lines': {'1': 'M2', '2': 'M3', '3': 'M4', '5': 'M5', '6': 'M6', '7': 'M7', '8': 'M8', '9': 'M1A', '10': 'M1B', '17': 'M9'}}

{'Group': 'T', 'Name': 'Tramvay Hatları', 'Lines': {'11': 'T1', '12': 'T3', '13': 'T4', '14': 'T5'}}

{'Group': 'F', 'Name': 'Füniküler Hatları', 'Lines': {'4': 'F1', '20': 'F4'}}

{'Group': 'TF', 'Name': 'Teleferik Hatları', 'Lines': {'15': 'TF1', '16': 'TF2'}}



## Istanbul Metro Stations

In [3]:
# GET Request
r = requests.get("https://api.ibb.gov.tr/MetroIstanbul/api/MetroMobile/GetStations")
print(r)
# Save Json Data
json_obj = r.json()

# Extract Data
metro_stn_data = []

for row in json_obj:

    lat=row['DetailInfo']['Latitude']
    long=row['DetailInfo']['Longitude']

    data_dict = {k : row[k] for k in row.keys() if k not in ('Id', 'DetailInfo')}
    data_dict.update({'Latitude':lat})
    data_dict.update({'Longitude':long})

    metro_stn_data.append(data_dict)

df = pd.DataFrame(metro_stn_data)

<Response [200]>


In [4]:
df['Latitude'] = df['Latitude'].astype("float64")
df['Longitude'] = df['Longitude'].astype("float64")

In [5]:
df['RailType'] = (
    df['LineName'].str.extract(r'^(\D+)\d+\D*$', expand=False)
    .map({"M" : "Metro", "T" : "Tramway", "F" : "Funicular", "TF" : "Cable Car"})
)

In [6]:
# Total 195 metro stations
len(df['Name'].unique())

195

In [7]:
(
    df[['LineId', 'LineName', 'RailType']]
    .drop_duplicates()
    .sort_values('LineId', ignore_index=True)
)

,LineId,LineName,RailType
0,1,M2,Metro
1,2,M3,Metro
2,3,M4,Metro
3,4,F1,Funicular
4,5,M5,Metro
5,6,M6,Metro
6,7,M7,Metro
7,8,M8,Metro
8,9,M1A,Metro
9,10,M1B,Metro


In [8]:
df["FunctionalCode"].str.extract(r'^\D{1,2}-\w+-(.+)-\d+$', expand=False).unique()

array(['S-H-I'], dtype=object)

In [10]:
df.to_csv("tr_istanbul_metro_stns.csv", index=False)